In [1]:
import tensorflow as tf

In [2]:
# import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255)
val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    "/content/drive/MyDrive/25_2 Summer 25/Vision Research /BDCoin VGG Codes/Dataset/Train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_data = val_gen.flow_from_directory(
    "/content/drive/MyDrive/25_2 Summer 25/Vision Research /BDCoin VGG Codes/Dataset/Valid",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

vgg = tf.keras.applications.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

model = tf.keras.Sequential([
    vgg,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(train_data, validation_data=val_data, epochs=1)


Found 6644 images belonging to 6 classes.
Found 1280 images belonging to 6 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


208/208 ━━━━━━━━━━━━━━━━━━━━ 2400s 11s/step - accuracy: 0.2438 - loss: 2.1360 - val_accuracy: 0.4125 - val_loss: 1.6589


In [4]:
model.save("/content/drive/MyDrive/25_2 Summer 25/Vision Research /BDCoin VGG Codes/vgg_model.h5")
print("Saved: vgg_model.h5")

Saved: vgg_model.h5


In [6]:
# Example using Keras ModelCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_filepath = '/content/drive/MyDrive/25_2 Summer 25/Vision Research /BDCoin VGG Codes/vgg_model_checkpoints/best_model.weights.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)
model.fit(train_data, validation_data=val_data, epochs=9, callbacks=[model_checkpoint_callback])
model.save("/content/drive/MyDrive/25_2 Summer 25/Vision Research /BDCoin VGG Codes/vgg_model.h5")
print("Saved: vgg_model.h5")

Epoch 1/9
208/208 ━━━━━━━━━━━━━━━━━━━━ 106s 507ms/step - accuracy: 0.2821 - loss: 1.7122 - val_accuracy: 0.4125 - val_loss: 1.7186
Epoch 2/9
208/208 ━━━━━━━━━━━━━━━━━━━━ 108s 515ms/step - accuracy: 0.2720 - loss: 1.7157 - val_accuracy: 0.4125 - val_loss: 1.6628
Epoch 3/9
208/208 ━━━━━━━━━━━━━━━━━━━━ 109s 520ms/step - accuracy: 0.2767 - loss: 1.7046 - val_accuracy: 0.4125 - val_loss: 1.6619
Epoch 4/9
208/208 ━━━━━━━━━━━━━━━━━━━━ 107s 514ms/step - accuracy: 0.2824 - loss: 1.7002 - val_accuracy: 0.4125 - val_loss: 1.6608
Epoch 5/9
208/208 ━━━━━━━━━━━━━━━━━━━━ 106s 507ms/step - accuracy: 0.2754 - loss: 1.7019 - val_accuracy: 0.4125 - val_loss: 1.6611
Epoch 6/9
208/208 ━━━━━━━━━━━━━━━━━━━━ 101s 486ms/step - accuracy: 0.2862 - loss: 1.6955 - val_accuracy: 0.4125 - val_loss: 1.6661
Epoch 7/9
208/208 ━━━━━━━━━━━━━━━━━━━━ 101s 486ms/step - accuracy: 0.2780 - loss: 1.6997 - val_accuracy: 0.4125 - val_loss: 1.6612
Epoch 8/9
208/208 ━━━━━━━━━━━━━━━━━━━━ 101s 486ms/step - accuracy: 0.2858 - loss: 1

Saved: vgg_model.h5


In [8]:
test_gen = ImageDataGenerator(rescale=1./255)

test_data = test_gen.flow_from_directory(
    "/content/drive/MyDrive/25_2 Summer 25/Vision Research /BDCoin VGG Codes/Dataset/Test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False    # IMPORTANT!
)

class_names = list(test_data.class_indices.keys())
num_classes = len(class_names)

Found 798 images belonging to 6 classes.


In [9]:
import numpy as np

from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

#Predicting test dataset

pred_probs = model.predict(test_data)
pred_classes = np.argmax(pred_probs, axis=1)
true_classes = test_data.classes

#Genearting Classification Report
report = classification_report(true_classes, pred_classes, target_names=class_names)
print(report)

with open("/content/drive/MyDrive/25_2 Summer 25/Vision Research /BDCoin VGG Codes/classification_report.txt", "w") as f:
    f.write(report)

print("Saved: classification_report.txt")


#Generating confusion matrix and heatmap
cm = confusion_matrix(true_classes, pred_classes)

plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")

plt.savefig("/content/drive/MyDrive/25_2 Summer 25/Vision Research /BDCoin VGG Codes/confusion_matrix.png", dpi=300)
plt.close()

print("Saved: confusion_matrix.png")

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 210s 9s/step
              precision    recall  f1-score   support

          1b       0.00      0.00      0.00        97
          1f       0.00      0.00      0.00       129
          2b       0.00      0.00      0.00        83
          2f       0.33      1.00      0.50       265
          5b       0.00      0.00      0.00       100
          5f       0.00      0.00      0.00       124

    accuracy                           0.33       798
   macro avg       0.06      0.17      0.08       798
weighted avg       0.11      0.33      0.17       798

Saved: classification_report.txt


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Saved: confusion_matrix.png
